In [9]:
import os
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

load_dotenv()

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# langchain支持deepseek以model identifier string方式声明（需要在.env中配置DEEPSEEK_API_KEY），考虑到调用成本，改用免费的model
# agent = create_agent(
#     model = "deepseek-chat",
#     tools = [get_weather],
#     system_prompt = "You are a helpful assistant",
# )

model = ChatOpenAI(
    model="qwen3-max",
    temperature=0.1,
    max_tokens=1000,
    timeout=30,
    api_key=os.getenv('API_KEY'),
    base_url=os.getenv('BASE_URL')
)
agent = create_agent(
    model,
    tools = [get_weather],
    system_prompt = "You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='d24e4416-c8fa-4973-a46d-449b26d31ee7'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 265, 'total_tokens': 286, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-0619e06e-345a-4b94-a25c-5d096036659b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--49f5b8ed-cf40-407c-89c9-49fe714fe843-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_711421614a844b6d851b616d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 265, 'output_tokens': 21, 'total_tokens': 286, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}),
  ToolMessage(content="It's always sunny in sf!", name='get_weath

## Agents

In [12]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain.tools import tool

class ContractInfo(BaseModel):
    name: str
    email: str
    phone: str

@tool
def search_tool():
    """检索工具"""
    print("It's a search tool.")

agent = create_agent(
    model=model,
    tools=[search_tool],
    response_format=ToolStrategy(ContractInfo),
)

result = agent.invoke({
    "messages":[{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
}
)

result["structured_response"]

ContractInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [13]:
from langchain.agents.structured_output import ProviderStrategy

agent = create_agent(
    model=model,
    response_format=ProviderStrategy(ContractInfo),
)

result = agent.invoke({
    "messages":[{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]

ContractInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')